# Spark DataFrame Basics

Spark DataFrames are the workhouse and main way of working with Spark and Python post Spark 2.0. DataFrames act as powerful versions of tables, with rows and columns, easily handling large datasets. The shift to DataFrames provides many advantages:
* A much simpler syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs
    
If you've used R or even the pandas library with Python you are probably already familiar with the concept of DataFrames. Spark DataFrame expand on a lot of these concepts, allowing you to transfer that knowledge easily by understanding the simple syntax of Spark DataFrames. Remember that the main advantage to using Spark DataFrames vs those other programs is that Spark can handle data across many RDDs, huge data sets that would never fit on a single computer. That comes at a slight cost of some "peculiar" syntax choices, but after this course you will feel very comfortable with all those topics!

Let's get started!

## Creating a DataFrame

First we need to start a SparkSession:

In [2]:
from pyspark.sql import SparkSession

Then start the SparkSession

In [4]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()

You will first need to get the data from a file (or connect to a large distributed file like HDFS, we'll talk about this later once we move to larger datasets on AWS EC2).

In [6]:
# We'll discuss how to read other options later.
# This dataset is from Spark's examples

# Might be a little slow locally
df = spark.read.json('people.json')

#### Showing the data

In [8]:
# Note how data is missing!
df.show()

In [9]:
df.printSchema()

In [10]:
df.columns

In [11]:
df.describe()

Some data types make it easier to infer schema (like tabular formats such as csv which we will show later). 

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [13]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Next we need to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [15]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), nullable=True)]

In [16]:
final_struc = StructType(fields=data_schema)

In [17]:
df = spark.read.json('people.json', schema=final_struc)

In [18]:
df.printSchema()

### Grabbing the data

In [20]:
df['age']

In [21]:
type(df['age'])

In [22]:
df.select('age')

In [23]:
type(df.select('age'))

In [24]:
df.select('age').show()

In [25]:
# Returns list of Row objects
df.head(2)

Multiple Columns:

In [27]:
df.select(['age','name'])

In [28]:
df.select(['age','name']).show()

### Creating new columns

In [30]:
# Adding a new column with a simple copy
df.withColumn('newage',df['age']).show()

In [31]:
df.show()

In [32]:
# Simple Rename
df.withColumnRenamed('age','supernewage').show()

More complicated operations to create new columns

In [34]:
df.withColumn('doubleage',df['age']*2).show()

In [35]:
df.withColumn('add_one_age',df['age']+1).show()

In [36]:
df.withColumn('half_age',df['age']/2).show()

In [37]:
df.withColumn('half_age',df['age']/2)

We'll discuss much more complicated operations later on!

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [40]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [41]:
sql_results = spark.sql("SELECT * FROM people")

In [42]:
sql_results

In [43]:
sql_results.show()

In [44]:
spark.sql("SELECT * FROM people WHERE age=30").show()

We won't really be focusing on using the SQL syntax for this course in general, but keep in mind it is always there for you to get you out of bind quickly with your SQL skills!

Alright that is all we need to know for now!